## Spotipy 이탈 예측 모델 학습 가이드 (팀원용)

이 노트북은 **팀원 한 명당 1개씩 모델을 맡아 학습/실험**하고, 결과를 서로 비교할 수 있도록 돕는 사용 설명서입니다. 
코드 내용은 백엔드 담당자가 이미 설정해 두었으므로, **팀원도 아래 단계만 따르면 실험을 반복**할 수 있습니다.

---

### 1. 전체 흐름 한눈에 보기

- **데이터 준비**: `data/enhanced_data_not_clean_FE_delete.csv`
- **전처리 파이프라인**: `backend/preprocessing_pipeline.py` 의 `preprocess_and_split()`
  - 결측치 처리 → 이상치(IQR) 처리 → 수치 스케일링(StandardScaler) → 범주 원-핫인코딩 → Train/Test 분리
- **모델 정의·선택**: `backend/models.py` 의 `MODEL_REGISTRY`, `get_model()`
- **실험 스크립트**: `backend/train_experiments.py`
  - 전처리 호출 → 모델 학습 → F1 / AUC / PR-AUC / Precision/Recall / 혼동행렬 계산 → `models/metrics.json` 에 결과 자동 저장
- **공통 설정 파일**: `backend/config.py`
  - 데이터 경로, 테스트 비율, 랜덤 시드, 사용할 모델 이름, threshold 범위, 메트릭 저장 위치 등

즉, **팀원은 주로 `config.py`에서 모델 이름만 바꾸고, `train_experiments.py`만 실행**하면 됩니다.

---

### 2. 실험 실행 순서 (팀원용 Step-by-step)

#### 2-1. 사전 준비

- **1) 프로젝트 폴더로 이동**
  - VS Code / Cursor 터미널에서 프로젝트 루트로 이동합니다.
  - 예시 (Windows PowerShell 기준):

```bash
cd C:\project\SKN21-2nd-2Team
```

- **2) 필요한 패키지 설치 (최초 1회)**

```bash
pip install -r requirements.txt
```

#### 2-2. 자신이 맡은 모델 이름 정하기

- **`backend/models.py`** 를 엽니다.
- 파일 상단에 `MODEL_REGISTRY` 라는 딕셔너리가 있습니다.
- 예시 (일부):

```python
MODEL_REGISTRY: Dict[ModelName, ModelSpec] = {
    "rf": ModelSpec(
        cls=RandomForestClassifier,
        default_params={
            "n_estimators": 300,
            "max_depth": None,
            "min_samples_split": 5,
            "class_weight": "balanced",
            "n_jobs": -1,
        },
    ),
    "logit": ModelSpec(
        cls=LogisticRegression,
        default_params={
            "max_iter": 1000,
            "n_jobs": -1,
        },
    ),
}
```

- **각 팀원은 이 키 값 중 하나를 맡아 실험**합니다.
  - 예: 팀원 A → `"rf"`, 팀원 B → `"logit"`
- 새 모델(XGB, LGBM 등)을 추가하고 싶다면, **ML 담당자와 상의해서 `MODEL_REGISTRY`에 항목을 추가**하면 됩니다.

#### 2-3. 사용할 모델 선택 (config 수정)

- **`backend/config.py`** 를 엽니다.
- 아래 부분에서 **기본 모델 이름**을 확인/수정합니다.

```python
DEFAULT_MODEL_NAME: str = "rf"  # "rf", "logit" 등 models.py의 MODEL_REGISTRY 키
```

- 각자 실험할 때, **자신이 맡은 모델 이름으로만 이 값을 수정**하고 실행하면 됩니다.
  - 예: 로지스틱 회귀를 실험할 팀원이라면 `"logit"` 으로 변경.

> 협업 팁: 
> - 공용 브랜치에 커밋할 때는 `DEFAULT_MODEL_NAME`를 다시 기본값(예: `"rf"`)로 맞추거나, 
> - 각자 로컬에서만 수정한 뒤 결과만 공유하는 방식으로 운영하는 게 좋습니다.

#### 2-4. 실험 실행 (학습 + 평가)

- 프로젝트 루트에서 아래 명령을 실행합니다.

```bash
python -m backend.train_experiments
```

- 실행되면 터미널에 다음과 같은 정보가 출력됩니다.
  - **F1 Score**
  - **ROC-AUC** (AUC)
  - **PR-AUC** (Precision-Recall AUC)
  - **Best Threshold** (F1이 가장 좋은 임계값)
  - **Precision / Recall (해당 threshold 기준)**
  - **Confusion Matrix (TN, FP, FN, TP)**
  - 학습/테스트 데이터 개수 (n_train, n_test)

- 동시에, 결과가 **`models/metrics.json`** 파일에 자동으로 누적 저장됩니다.
  - 여러 번 실행하면 리스트 형태로 계속 append 됩니다.

#### 2-5. 결과 비교하기

- `models/metrics.json` 파일을 열어보면, 실행할 때마다 하나의 dict가 쌓여 있습니다.
- 예시(요약):

```json
[
  {
    "model": "rf",
    "f1": 0.87,
    "auc": 0.93,
    "pr_auc": 0.90,
    "best_threshold": 0.18,
    "precision": 0.85,
    "recall": 0.89,
    "confusion_matrix": {"tn": 1000, "fp": 100, "fn": 80, "tp": 420},
    "n_train": 6400,
    "n_test": 1600,
    "data_path": "data/enhanced_data_not_clean_FE_delete.csv",
    "test_size": 0.2,
    "random_state": 42,
    "timestamp": "2025-11-22T15:30:01"
  }
]
```

- 팀원별로 **`model` 이름을 기준으로 F1, AUC, PR-AUC, Precision/Recall, Confusion Matrix**를 비교하면서 
  어떤 모델이 이탈 예측에 더 잘 맞는지 토론하면 됩니다.

---

### 3. 머신러닝 실행 파일 설명

#### 3-1. `backend/preprocessing_pipeline.py`

- **역할**: 원본 CSV를 받아서 **모델이 바로 쓸 수 있는 형태의 X_train, X_test, y_train, y_test**로 변환하는 전처리 파이프라인.
- 핵심 함수:
  - **`clean_missing_values(df)`**: 결측치 처리
    - `listening_time`, `songs_played_per_day` → 중앙값(median)
    - `payment_failure_count`, `app_crash_count_30d` → 0
    - `customer_support_contact`, `promotional_email_click` → False
  - **`handle_outliers_iqr(df)`**: IQR 기반 이상치 clip
  - **`build_preprocessor(df)`**: 
    - 숫자 컬럼 → median imputer + `StandardScaler`
    - 범주 컬럼(`gender`, `device_type`, `subscription_type`, `country`) → most_frequent imputer + `OneHotEncoder(handle_unknown="ignore")`
  - **`preprocess_and_split(path, test_size, random_state)`**:
    - 위 과정을 모두 포함해서 **하나의 함수로 호출**할 수 있게 만든 메인 함수
    - 반환: `X_train_processed, X_test_processed, y_train, y_test, preprocessor`
  - **`save_processed_data(...)` / `load_processed_data(...)`**:
    - 전처리된 데이터와 `preprocessor` 객체를 `.pkl`로 저장/로딩하는 유틸

팀원은 이 파일을 직접 수정할 필요는 거의 없고, 
**“원본 CSV를 알아서 잘 전처리해준다” 정도로 이해**하고 있으면 됩니다.

#### 3-2. `backend/models.py`

- **역할**: 문자열로 된 모델 이름을 받아서 **실제 sklearn 모델 객체를 만들어 주는 팩토리**.
- 주요 구성:
  - `MODEL_REGISTRY`: 모델 이름 → (클래스, 기본 하이퍼파라미터) 매핑
  - `ModelSpec`: 모델 설정을 담는 데이터클래스
  - `get_model(name, random_state, **overrides)`:
    - 예: `get_model("rf", random_state=42, n_estimators=500)`
    - registry에서 기본 설정을 읽고, 랜덤 시드를 넣고, 필요한 경우 override 적용 후 모델 인스턴스 생성

- 새 모델을 넣고 싶을 때는 **`MODEL_REGISTRY`에 항목을 추가**하면 됩니다.
  - 팀원은 주로 **`name`만 바꿔서 다른 모델을 써본다**고 생각하면 됩니다.

#### 3-3. `backend/config.py`

- **역할**: 실험에서 자주 바꾸는 값들을 한 곳에 모아 둔 설정 파일.
- 주요 상수:
  - **`DATA_PATH`**: 사용할 CSV 경로
  - **`TEST_SIZE`**: 테스트 데이터 비율 (예: 0.2 → 20%)
  - **`RANDOM_STATE`**: 랜덤 시드(재현성용)
  - **`DEFAULT_MODEL_NAME`**: 기본으로 사용할 모델 이름
  - **`THRESH_START`, `THRESH_END`, `THRESH_STEP`**: threshold 스캔 범위
  - **`METRICS_PATH`**: 메트릭 저장 파일 경로 (`models/metrics.json`)

팀원은 보통 **`DEFAULT_MODEL_NAME` 정도만 바꿔도 충분**하며, 
나머지 값들은 팀 합의 후 ML 담당자가 조정하는 것을 추천합니다.

#### 3-4. `backend/train_experiments.py`

- **역할**: 
  - 전처리 호출 → 모델 학습 → 여러 지표 계산 → 결과를 콘솔 + JSON 파일에 저장하는 **실험 메인 스크립트**.
- 주요 흐름:
  1. `preprocess_and_split()` 호출 → `X_train, X_test, y_train, y_test` 생성
  2. `get_model(MODEL_NAME, random_state)`로 모델 생성
  3. `model.fit(X_train, y_train)` 학습
  4. `predict_proba`로 예측 확률 계산
  5. 여러 threshold를 돌면서 **F1이 가장 좋은 지점** 찾기
  6. 해당 threshold에서 **Precision/Recall, Confusion Matrix** 계산
  7. ROC-AUC, PR-AUC 계산
  8. `save_metrics(...)`로 `models/metrics.json`에 결과 누적 저장

---

### 4. pkl 파일 설명

`.pkl`(pickle) 파일은 **파이썬 객체를 그대로 저장한 파일 형식**입니다. 
엑셀처럼 바로 열어보는 용도라기보다는, **파이썬 코드에서 다시 불러와서 사용하는 용도**입니다.

#### 4-1. 현재 파이프라인 기준 주요 pkl

- **`data/X_train_processed.pkl`, `data/X_test_processed.pkl`**
  - 전처리된 학습/테스트 입력 데이터 (`numpy` 배열 또는 희소 행렬 형태)
  - `preprocessing_pipeline.py`의 `save_processed_data()`로 생성 가능
- **`data/y_train.pkl`, `data/y_test.pkl`**
  - 타깃(label) 벡터 (0/1 이탈 여부)
- **`data/preprocessor.pkl`**
  - `ColumnTransformer` 전처리기 객체
  - 나중에 새로운 데이터(실제 유저 입력)에 같은 전처리를 적용할 때 재사용

#### 4-2. `data/processed/` 폴더의 pkl들

- `data/processed/X_train.pkl`, `X_test.pkl`, `y_train.pkl`, `y_test.pkl`, `scaler.pkl`
  - **이전 버전 파이프라인에서 사용하던 결과물**입니다.
  - 현재 학습 코드(`preprocessing_pipeline.py`, `train_experiments.py`, `config.py`)에서는 이 경로를 **참조하지 않습니다.**
  - 과거 실험 결과를 따로 보관할 필요가 없다면, **정리 차원에서 삭제해도 무방**합니다.
  - 다만, 혹시 몰라 백업을 남기고 싶다면 `backup/` 같은 폴더로 옮겨 두고 관리해도 좋습니다.

---

### 5. 지표(F1, AUC 등) 해석 간단 정리

- **F1 Score**
  - Precision(정밀도)과 Recall(재현율)의 조화 평균
  - 값이 1에 가까울수록 좋고, **양성(이탈 고객) 탐지 성능 전반**을 하나의 숫자로 보고 싶을 때 사용
- **ROC-AUC**
  - 모든 threshold에서의 TPR(FP 대비) 성능을 요약한 값
  - 0.5는 랜덤, 1.0에 가까울수록 좋음
- **PR-AUC**
  - Precision-Recall 곡선 아래 면적
  - **양성 비율이 적은 불균형 데이터**에서 특히 의미가 큼
- **Precision (정밀도)**
  - 모델이 "이탈"이라고 예측한 것 중 실제로 이탈인 비율
- **Recall (재현율)**
  - 실제 이탈 고객 중에서 모델이 얼마나 잘 찾아냈는지
- **Confusion Matrix (TN, FP, FN, TP)**
  - TN: 정상 고객을 정상이라고 맞춘 수
  - FP: 정상 고객을 이탈이라고 잘못 예측한 수
  - FN: 이탈 고객을 정상이라고 잘못 예측한 수
  - TP: 이탈 고객을 이탈이라고 맞춘 수

팀 상황에 따라, 
- "FP가 너무 많으면 마케팅 비용 낭비"인지, 
- "FN이 많으면 이탈 고객을 놓쳐서 손실"인지 등을 고려해서 
**어떤 지표를 더 중요하게 볼지 팀 내에서 합의**하면 좋습니다.

---

### 6. 정리

- **팀원은 보통 아래만 기억하면 충분합니다.**
  - `config.py`에서 `DEFAULT_MODEL_NAME`를 자신의 모델로 바꾸기
  - 터미널에서 `python -m backend.train_experiments` 실행하기
  - 터미널 출력과 `models/metrics.json`을 보고 팀원들과 성능 비교하기
- 나머지 전처리/모델 구성/설정 관리는 **데이터/ML 담당자가 중심이 되어 유지**하면 됩니다.

궁금한 점이나 오류가 나면, 
- 어떤 명령을 실행했는지,
- 에러 메시지 전체를 캡처해서 팀 채팅에 공유하면 
 빠르게 같이 디버깅할 수 있습니다.


### 7. 성능 우선순위 & 모델 배정 예시

#### 7-1. 성능 지표 우선순위 (추천)

이 프로젝트에서는 **"이탈 고객을 잘 잡아내는 것"**이 중요하면서도, 
너무 과한 오탐(FP)도 피해야 합니다. 그래서 지표를 아래 순서로 보는 것을 추천합니다.

1. **F1 Score (1순위)**
   - Precision(정밀도)과 Recall(재현율을 동시에 고려한 지표입니다.
   - 이탈 고객을 잘 찾으면서도, 예측이 너무 엉망이지는 않은지 한 번에 보기 좋습니다.
2. **PR-AUC (2순위)**
   - 불균형 데이터에서 **양성(이탈)**에 대한 전반적인 탐지 성능을 잘 보여줍니다.
   - 동일 F1이라도 PR-AUC가 높은 모델이 **threshold를 조정했을 때 더 유연하게 운영**될 가능성이 있습니다.
3. **Recall (3순위, 상황에 따라 중요)**
   - 실제 이탈 고객 중 몇 %를 잡아내는지가 중요하다면, Recall을 따로 챙겨 봅니다.
   - "이탈자를 놓치는 것(FN)이 더 아프다"고 팀에서 합의하면, Recall에 가중치를 둘 수 있습니다.
4. **ROC-AUC (참고용)**
   - 전체 구분 능력을 보여주는 지표라 참고로 좋지만, 
     PR-AUC와 F1이 이미 좋다면 ROC-AUC는 서브로만 봐도 충분합니다.
5. **Confusion Matrix (TN, FP, FN, TP)**
   - 나중에 "FP가 많아서 마케팅 비용이 너무 든다", "FN이 많아 이탈자를 놓친다" 같은 논의에 필수입니다.
   - 어떤 모델이 **어떤 실수를 많이 하는지**를 직관적으로 보여줍니다.

> **정리:**
> - 기본 비교는 **F1 → PR-AUC → Recall** 순으로 보고, 
> - ROC-AUC와 Confusion Matrix는 **보조적인 해석용**으로 활용하는 것을 권장합니다.

---

#### 7-2. 5인 팀 기준 모델 배정 예시 (추천 5개 모델)

위 지표 우선순위를 기준으로, 이 데이터(`enhanced_data_not_clean_FE_delete.csv`)에 잘 맞는 **5개 메인 모델**을 아래처럼 나누어 실험하는 것을 추천합니다.

> **추천 5개 메인 모델**
> - `"rf"`   : RandomForestClassifier  
> - `"xgb"`  : XGBClassifier  
> - `"lgbm"` : LGBMClassifier  
> - `"et"`   : ExtraTreesClassifier  
> - `"logit"`: LogisticRegression

아래는 **5명이 서로 다른 모델을 맡아서 실험**할 때의 한 가지 예시입니다. 
(실제 배정은 팀에서 자유롭게 바꿔도 됩니다.)

| 팀원 | 역할/모델 타입                 | 예시 모델                     | 비고 |
|------|--------------------------------|-------------------------------|------|
| 1    | **Baseline 트리 앙상블**      | RandomForest (`"rf"`)        | 기준 트리 앙상블, 전체 비교 기준선 |
| 2    | **Baseline 선형 모델**        | Logistic Regression (`"logit"`) | 해석력이 좋아, 변수 영향 설명에 유리 |
| 3    | **Gradient Boosting 계열 1**  | XGBoost (`"xgb"`)            | 부스팅 대표 1, 성능 기대치 높음 |
| 4    | **Gradient Boosting 계열 2**  | LightGBM (`"lgbm"`)          | 부스팅 대표 2, 속도/성능 밸런스 좋음 |
| 5    | **트리 앙상블 변형**          | ExtraTrees (`"et"`)          | RF와 다른 성향의 트리 앙상블, 비교용 |

- 위 5개 모델을 먼저 한 번씩 돌려보고, 
  - **가장 성능이 좋은 모델 1개를 골라 2라운드에서 하이퍼파라미터 튜닝 담당**이 집중적으로 다듬는 흐름을 추천합니다.
- 비교적 구현이 쉬운 모델(`"rf"`, `"logit"`, `"et"`)을 맡은 팀원은, 
  - 시간이 허용되면 **추가로 1개 모델(KNN, HistGradientBoosting 등)을 더 실험해 보는 "보너스 과제"**로 가져가도 좋습니다.
